In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
train_data=np.array(pd.read_csv('ProcessedTrain.csv',delimiter=','))[:,2:]

test_data=np.array(pd.read_csv('ProcessedTest.csv',delimiter=','))[:,2:]
id=np.array(pd.read_csv('ProcessedTest.csv',delimiter=','))[:,1]

In [3]:
train_data.shape

(1460, 80)

In [4]:
X_train = train_data[:,:79]
y_train = train_data[:, 79]

In [5]:
y_train

array([208500., 181500., 223500., ..., 266500., 142125., 147500.])

In [54]:
def build_model():
  model = keras.Sequential([
    keras.layers.Dense(64, activation=tf.nn.relu, kernel_regularizer=keras.regularizers.l2(0.01),
                       input_shape=(X_train.shape[1],)),
    keras.layers.Dense(64, kernel_regularizer=keras.regularizers.l2(0.01), activation=tf.nn.relu),
    keras.layers.Dense(1)
  ])

  optimizer = tf.train.AdamOptimizer(0.01)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae'])
  return model

model = build_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 64)                5120      
_________________________________________________________________
dense_25 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_26 (Dense)             (None, 1)                 65        
Total params: 9,345
Trainable params: 9,345
Non-trainable params: 0
_________________________________________________________________


In [55]:
# Display training progress by printing a single dot for each completed epoch
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')

EPOCHS = 550

# Store training stats
history = model.fit(X_train, y_train, epochs=EPOCHS,
                    validation_split=0.2, verbose=0,
                    callbacks=[PrintDot()])


....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
..................................................

In [56]:
test_predictions = model.predict(test_data).flatten()

In [57]:
d={'Id':id.astype(np.int32),'SalePrice':test_predictions}

In [58]:
df=pd.DataFrame(d)

In [59]:
df.to_csv("predictions.csv",index=False)